# Unit Commitment

In [1]:
import numpy as np

import andes
import ams

import pandas as pd

import json

import cvxpy as cp

In [2]:
ams.__version__

'0.6.5.post72.dev0+g3c5fde5'

In [3]:
ams.config_logger(stream_level=10)

In [4]:
sp = ams.load(ams.get_case('ieee14/ieee14_uc.xlsx'),
              setup=True,
              )

Input format guessed as xlsx.
Parsing input file "/home/jwang/work/ams/ams/cases/ieee14/ieee14_uc.xlsx"...
Input file parsed in 0.2424 seconds.
Adjust bus index to start from 0.
System set up in 0.0141 seconds.


In [ ]:
sp.UC.syms.generate_symbols()

In [ ]:
sp.UC.setup(show_code=True)

In [ ]:
sp.UC.run()

In [ ]:
sp.UC.obj.e_str

In [ ]:
sp.UC.ug.shape